In [3]:
import sys
sys.path.append("/projects/b1042/GoyalLab/jaekj/SCTAB_FINAL")
from Vanilla_diff_model import ConditionalDenoiser, DiffusionModule, sigmoid_schedule

ModuleNotFoundError: No module named 'dask'

In [1]:
import importlib.util
import sys
from pathlib import Path

file_path = "/projects/b1042/GoyalLab/jaekj/SCTAB_FINAL/scTab-devel_old/notebooks/model_evaluation/Vanilla_diff_model.py"
module_name = "Vanilla_diff_model"

spec = importlib.util.spec_from_file_location(module_name, file_path)
module = importlib.util.module_from_spec(spec)
sys.modules[module_name] = module
spec.loader.exec_module(module)

ConditionalDenoiser = module.ConditionalDenoiser

/projects/b1042/GoyalLab/jaekj/python/DL_py12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'dask'

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import os
import scanpy as sc
import scipy as sp

In [ ]:
embedding_path = '/projects/b1042/GoyalLab/jaekj/SCTAB_FINAL/Embedding/adatas'
train_file = os.path.join(embedding_path, 'train_emb_tabnet.h5ad')
val_file   = os.path.join(embedding_path, 'val_emb_tabnet.h5ad')
test_file  = os.path.join(embedding_path, 'test_emb_tabnet.h5ad')

# Load the data
train_adata = sc.read_h5ad(train_file)
val_adata   = sc.read_h5ad(val_file)
test_adata  = sc.read_h5ad(test_file)

# Check shapes
print("Train shape:", train_adata.shape)
print("Validation shape:", val_adata.shape)
print("Test shape:", test_adata.shape)

# Extract embedding
def to_dense(x):
    if sp.issparse(x):
        return x.toarray()
    return x

X_train_emb = torch.tensor(to_dense(train_adata.X), dtype=torch.float32)
X_val_emb   = torch.tensor(to_dense(val_adata.X), dtype=torch.float32)
X_test_emb  = torch.tensor(to_dense(test_adata.X), dtype=torch.float32)

y_train_emb = torch.tensor(train_adata.obs["y_true"].values, dtype=torch.long)
y_val_emb   = torch.tensor(val_adata.obs["y_true"].values, dtype=torch.long)
y_test_emb  = torch.tensor(test_adata.obs["y_true"].values, dtype=torch.long)

In [ ]:
T = 1000
beta_min=1e-5
beta_max=0.023
low = -6
high = 6

In [ ]:
# --- Initialize model architecture exactly as during training ---
uniques = torch.unique(y_train_emb)
diff_model = ConditionalDenoiser(
    latent_dim=X_train_emb.shape[1],
    num_classes=len(uniques),
    T=T,
    label_emb_dim=64
)

# Set beta schedule
diff_model.betas = sigmoid_schedule(T=T, beta_min=beta_min, beta_max=beta_max, low=low, high=high)

# Wrap in Lightning module
diffusion_module = DiffusionModule(
    model=diff_model,
    T=T,
    betas=diff_model.betas,
)

# Load trained weights
ckpt_path = "/projects/b1042/GoyalLab/jaekj/SCTAB_FINAL/diffusion_module_state_wo_yemb.pt"
diffusion_module.load_state_dict(torch.load(ckpt_path, map_location=device))
diffusion_module.to(device).eval()

In [ ]:
@torch.no_grad()
def ddpm_sample(diffusion_module, y_label, shape, device):
    """
    diffusion_module: trained DiffusionModule
    y_label: tensor of shape [batch_size] with class labels
    shape: (batch_size, latent_dim)
    """
    T = diffusion_module.T
    betas = diffusion_module.betas
    alphas = 1 - betas
    alphas_bar = torch.cumprod(alphas, dim=0)

    x_t = torch.randn(shape, device=device)  # start from noise

    for t_inv in reversed(range(T)):
        t = torch.full((shape[0],), t_inv, device=device, dtype=torch.long)
        pred_noise = diffusion_module(x_t, t, y_label)

        alpha_t = alphas[t_inv]
        alpha_bar_t = alphas_bar[t_inv]

        if t_inv > 0:
            beta_t = betas[t_inv]
            noise = torch.randn_like(x_t)
        else:
            noise = 0

        # DDPM update step
        x_t = (1 / torch.sqrt(alpha_t)) * (x_t - ((1 - alpha_t) / torch.sqrt(1 - alpha_bar_t)) * pred_noise) + torch.sqrt(beta_t) * noise

    return x_t

In [ ]:
# --- DDPM sampling ---
batch_size = 32
y_label = torch.randint(0, len(uniques), (batch_size,), device=device)

samples = ddpm_sample(
    diffusion_module,
    y_label,
    shape=(batch_size, X_train_emb.shape[1]),
    device=device
)

print(samples.shape)  # [batch_size, latent_dim]